### Import necessary libraries for file extraction (IBM)

In [2]:
import wget
import os
import cv2
import shutil
import glob
import numpy as np


## Download Files To Local Machine using credentials (DARWIN's)

In [ ]:
credentials = {
    'IBM_API_KEY_ID': '3ifDMUyTxwW7C30JlKVRdi5FpijnReisfUVLNwjPgFBj',
    'IAM_SERVICE_ID': 'crn:v1:bluemix:public:iam-identity::a/36c5a36d7ceb4e7690fc654b3802a2f9::serviceid:ServiceId-f43d7268-4c52-4dbd-b57b-d54ff7c7b1f2',
    'ENDPOINT': 'https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'obj-detect-facemasks',
    'FILE': '0e801b96-e694-48d3-891a-d5443c2124ea.jpg'
}

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

### Download files

In [ ]:
# cos.download_file(Bucket='bucketods-test',Key='mnist_test.csv',Filename='data/mnist_test2.csv')

### Configuration

In [3]:
CLEAN = False  # USE WITH CARE! Wipe out saved files when this is true (else reuse for speed)
FRAMES_DIR = "frames"  # Output dir to hold/cache the original frames
OUTPUT_DIR = "output"

## Create or clean the directories
Caching the frames and output directories allows the processing to continue where it left off. This is particularly useful when using a shared system with deployment time limits. This also allows you to quickly `Run all` when tweaking Python code that does not affect the inference.

If you change the input video or just want a fresh start, you should `CLEAN` or change the directory names.

In [4]:
if CLEAN:
    if os.path.isdir(FRAMES_DIR):
        shutil.rmtree(FRAMES_DIR)
    if os.path.isdir(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)

if not os.path.isdir(FRAMES_DIR):
    os.mkdir(FRAMES_DIR)
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

### Read video using directory

In [5]:
input_video = 'livestream videos/training_video.mp4'

### Parse and explode the video file into JPEGs

In [179]:
if os.path.isfile(input_video):
    video_capture = cv2.VideoCapture(input_video)
else:
    raise Exception("File %s doesn't exist!" % input_video)

total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Frame count estimate is %d" % total_frames)

num = 0

# while video_capture.get(cv2.CAP_PROP_POS_FRAMES) < video_capture.get(cv2.CAP_PROP_FRAME_COUNT):
while video_capture.get(cv2.CAP_PROP_POS_FRAMES) < 100:
          
    success, image = video_capture.read()
    
    if success:
        
        num = int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        print("Writing frame {num} of {total_frames}".format(
            num=num, total_frames=total_frames), end="\r")
        cv2.imwrite('{frames_dir}/camt{num:04d}.jpg'.format(
            frames_dir=FRAMES_DIR, num=num), image)
    else:
        # TODO: If this happens, we need to add retry code
        raise Exception('Error writing camt{num:04d}.jpg'.format(
            num=int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))))

print("\nWrote {num} frames".format(num=num))

FRAME_FPS = int(video_capture.get(cv2.CAP_PROP_FPS))
FRAME_WIDTH = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
FRAME_HEIGHT = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
ROI_YMAX = int(round(FRAME_HEIGHT * 0.75))  # Bottom quarter = finish line

print("Frame Dimensions: %sx%s" % (FRAME_WIDTH, FRAME_HEIGHT))

Frame count estimate is 1787
Writing frame 100 of 1787
Wrote 100 frames
Frame Dimensions: 960x540



### Pairwise Frames Subtraction

In [180]:
jpgCounter = len(glob.glob1(FRAMES_DIR,"*.jpg"))
i = 1
j = 2

while j < jpgCounter:

    img1 = cv2.imread('{frames_dir}/camt{num:04d}.jpg'.format(
            frames_dir=FRAMES_DIR, num=i))
    
    while j < jpgCounter:

        img2 = cv2.imread('{frames_dir}/camt{num:04d}.jpg'.format(
            frames_dir=FRAMES_DIR, num=j))

        overlap = cv2.absdiff(img1, img2)
        overlap_mean = np.array(overlap).mean()

        j+=1

        #Change this value to increase the threshold of the detected movement
        if int(overlap_mean) >= 12: 

            #Assign new i and j here
            i = j - 1
            j += 1
            break

        else:
            os.remove('{frames}/camt{num:04d}.jpg'.format(frames=FRAMES_DIR, num= (j-1)))


print('Frames Substraction Completed')    
        
            

Frames Substraction Completed


### Creating new bucket in Cloud Object Storage (COS)

In [ ]:
bucket = 'bucketods-livestream' #Name of bucket

In [ ]:
cos.create_bucket(Bucket=bucket)

### Uploading file to COS

In [ ]:
jpgCounter = len(glob.glob1(FRAMES_DIR,"*.jpg"))

for num in range(1, jpgCounter):
    cos.upload_file(Filename='frames/camt{num:04d}.jpg'.format(num = num),Bucket='bucketods-livestream', Key='camt{num:04d}.jpg'.format(num = num))
    print("Uploading file {num} of {jpgCounter}".format(
            num=num, jpgCounter=jpgCounter), end="\r")
    
print("Uploaded to: {bucket}".format(bucket = bucket))

### Get list of Buckets in COS

In [ ]:
cos_credentials={
  "apikey": "3ifDMUyTxwW7C30JlKVRdi5FpijnReisfUVLNwjPgFBj",
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 6c04872e-5bc4-431d-b608-28492aa4955f",
  "iam_apikey_name": "cloud-annotations-binding",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/36c5a36d7ceb4e7690fc654b3802a2f9::serviceid:ServiceId-f43d7268-4c52-4dbd-b57b-d54ff7c7b1f2",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/36c5a36d7ceb4e7690fc654b3802a2f9:ac1d32d4-e472-487b-86e8-d53aef97a86e::"
}

auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

cos = ibm_boto3.client('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
for bucket in cos.list_buckets()['Buckets']:
    print(bucket['Name'])